In [ ]:
# 要下載到環境的模組
!pip install reportlab
!pip install flask
!pip install pyngrok
!pip install easyocr
!pip install openai==0.28
!pip install git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [ ]:
# 將colab掛載到雲端硬碟上
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/ALL/ppt_program

/content/drive/MyDrive/ALL/ppt_program


In [ ]:
import time
import openai
import os
import threading
from flask import Flask, render_template, request, jsonify, url_for, send_from_directory, redirect
from pyngrok import ngrok
from pathlib import Path
import shutil
import easyocr
import re
import whisper
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import sys
sys.path.append('/content/drive/MyDrive/ALL/ppt_program')
from shot_ocr import screenshot, ocr
import os
import numpy as np
from PIL import Image
from moviepy.editor import VideoFileClip
#from sound import mp4_to_mp3, mp3_to_txt
from detect_with_cut import detect, move_txt_files, extract_images_from_txt
from text_output import modify_txt_file, cut_question, extract_summary_from_text, summary_to_question, modify_keytxt, generate_keyword


app = Flask(__name__)
openai.api_key = "your key"
port = 5000
ngrok.set_auth_token("你的ngrok_token")
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url,port))
app.config["BASE_URL"] = public_url

# 找靜態資料夾路徑的路由
@app.route('/static/<path:path>')
def send_static(path):
    static_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'static')
    return send_from_directory(static_dir, path)

# 設定暫存資料夾路徑
temp_folder = 'user_folder'
app.config['temp_folder'] = temp_folder
temp_dir = app.config['temp_folder']

# 建立暫存資料夾（如果不存在）
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

###whisper 程式碼
def mp4_to_mp3(video_path):
    mp4_files = [os.path.join(video_path, f) for f in os.listdir(video_path) if f.endswith('.mp4')]
    # 確保只有一個 .mp3 檔案存在
    if len(mp4_files) == 1:
        mp4_file = mp4_files[0]

        # 获取文件名和扩展名
        file_name, file_ext = os.path.splitext(mp4_file)
        # 新的文件名
        #new_file_name = file_name + "_copy" + file_ext
        try:
            # 复制文件
            #shutil.copy(mp4_file, new_file_name)
            #print("文件复制成功！新文件路径：", new_file_name)
            # MP3 文件路径
            mp3_file_path = os.path.join(video_path, "video_sound.mp3")
            # 執行轉換
            video_clip = VideoFileClip(mp4_file)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(mp3_file_path)
            audio_clip.close()
            video_clip.close()
            os.system("whisper video_sound.mp3")

        except Exception as e:
            print("音檔轉換出錯：", e)
    else:
        print("找不到或存在多個以 .mp4 結尾的檔案。")

def mp3_to_txt(video_path):
    # 載入模型
    model = whisper.load_model("medium")
    audio_files = [os.path.join(video_path, f) for f in os.listdir(video_path) if f.endswith('.mp3')]
    # 確保只有一個 .mp3 檔案存在
    if len(audio_files) == 1:
        mp3_file = audio_files[0]

        # 處理音頻檔案，指定繁體中文
        result = model.transcribe(mp3_file, prompt="請轉錄以下繁體中文的內容：")
        # 獲取轉錄的文本
        transcription_text = result["text"]
        # 印出結果
        #print(transcription_text)
        txt_file = os.path.join(video_path, "audio.txt")
        # 將轉錄結果保存到文本文件中
        with open(txt_file, "w", encoding="utf-8") as file:
            file.write(transcription_text)

        print("mp3 to txt success.")
    else:
        print("找不到或存在多個以 .mp3 結尾的檔案。")
#################################

@app.route('/')
def home():
    return render_template('Home.html')

@app.route('/run', methods=['POST'])
def runmodel():
    start_time = time.time() # Record start time
    # 檢查截圖目錄是否存在
    snapshot_dir = os.path.join(temp_dir, 'snapshots')
    os.makedirs(snapshot_dir, exist_ok=True)
    # 獲取上傳的影片文件
    video_file = request.files['video']

    # 將影片保存到暫存文件夾中
    video_path = os.path.join(temp_dir, video_file.filename)
    video_file.save(video_path)

    # 檢查 ppt_shot_folder 中是否有.mp4檔
    video_files = [f for f in os.listdir(temp_dir) if f.endswith('.mp4')]
    if not video_files:
        return jsonify({'error': 'No video file found in ppt_shot_folder'}), 400

    #######程式執行區塊########
    #Lin
    screenshot(video_path, snapshot_dir)
    ocr(snapshot_dir,temp_folder)

    #ray
    mp4_to_mp3(temp_dir)
    mp3_to_txt(temp_dir)

    #Nien
    detect(snapshot_dir)

    labels_folder = os.path.join(snapshot_dir, "labels")
    result_folder = os.path.join(snapshot_dir, "result")
    os.makedirs(result_folder, exist_ok=True)
    # 移動標籤文件
    move_txt_files(labels_folder, snapshot_dir)

    # 提取特定標籤的圖片
    specified_label = 1
    for txt_file in os.listdir(snapshot_dir):
        if txt_file.endswith(".txt"):
            txt_file_path = os.path.join(snapshot_dir, txt_file)
            extract_images_from_txt(txt_file_path, specified_label, result_folder)

    jpg_folder = '/content/drive/MyDrive/ALL/ppt_program/static/ppt_images'
    # Move .jpg files from source folder to destination folder
    for file in os.listdir(result_folder):
        if file.endswith(".jpg"):
            src_file = os.path.join(result_folder, file)
            dest_file = os.path.join(jpg_folder, file)
            shutil.move(src_file, dest_file)
            print(f"Moved {src_file} to {dest_file}")

    end_time = time.time() # Record end time
    print("Execution time :", (end_time - start_time)/60, "min") # Print execution time
    return render_template('Home.html', success=True)

@app.route("/generate", methods=["GET", "POST"])
def generate():
    summary = None

    txt_files = [f for f in os.listdir(temp_dir) if f.endswith('.txt')]
    texts = []
    for file_name in txt_files:
        file_path = os.path.join(temp_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
          file_content = file.read()
          texts.append(file_content)

    # Combine the content of the two files
    combined_text = "\n\n".join(texts)
    # Extract summary
    summary = extract_summary_from_text(combined_text)
    with open(os.path.join(temp_dir, 'summary.txt'), 'w') as f:
        f.write(summary)

    #question.txt
    question_folder = os.path.join(temp_dir, "question_cut")
    os.makedirs(question_folder, exist_ok=True)

    summary_file = os.path.join(temp_dir, 'summary.txt')
    original_question = os.path.join(temp_dir, 'question.txt')
    question = summary_to_question(summary_file)
    with open(original_question, 'w') as f:
        f.write(question)

    final_question = os.path.join(temp_dir, 'change_question.txt')
    modify_txt_file(original_question, final_question)
    cut_question(final_question, question_folder)

    #keywords.txt
    keyword = generate_keyword(summary_file)
    with open(os.path.join(temp_dir, 'keywords.txt'), 'w') as f:
        f.write(keyword)

    final_keyword = os.path.join(temp_dir,'change_keywords.txt')
    modify_keytxt(os.path.join(temp_dir, 'keywords.txt'),final_keyword)

    return render_template("Page.html")

@app.route('/summary')
def summary():
    # 讀取圖片列表
    result_folder = '/content/drive/MyDrive/ALL/ppt_program/static/ppt_images'
    result_images = os.listdir(result_folder)

    return render_template("Summary.html", images=result_images)

# 顯示summry.txt的route
@app.route('/get_summary')
def get_summary():
    return send_from_directory(temp_dir, 'summary.txt')

def read_questions(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        questions = []
        options = {'A': [], 'B': [], 'C': [], 'D': []}
        answers = []
        explanation = []
        current_question = None
        for line in lines:
            line = line.strip()
            if line.startswith('Answer:'):
                answers.append(line.split(':')[1].strip())
            elif line.startswith('Explanation:'):
                explanation.append(line.split(':')[1].strip())
            elif line.startswith('A') or line.startswith('B') or line.startswith('C') or line.startswith('D'):
                options[line[0]].append(line[3:])
            elif line:
                if current_question is not None:
                    questions.append((current_question, options))
                    options = {'A': [], 'B': [], 'C': [], 'D': []}
                current_question = line
        if current_question is not None:
            questions.append((current_question, options))
    return questions, answers, explanation

#顯示問題的路由
@app.route('/<int:question_number>')
def display_question(question_number):
    question_folder = os.path.join(temp_dir, "question_cut")
    question_file = os.path.join(question_folder, f'question_{question_number}.txt')
    if os.path.exists(question_file):
        questions, answers, explanation = read_questions(question_file)
        return render_template('question.html', questions=questions, answers=answers, explanation=explanation, next_question=question_number + 1)
    else:
        # If the question file doesn't exist, redirect to the first question
        return redirect(url_for('display_question', question_number=1))
#顯示question頁面
@app.route('/question')
def question():
    # Redirect to the first question
    return redirect(url_for('display_question', question_number=1))

@app.route('/quize')
def quize():
    return render_template('question.html')

def read_keyword(file_path):
    keyword_explanation = {}
    with open(file_path, 'r') as file:
        for line in file:
            if ':' in line:
                delimiter = ':'
                keyword, explanation = line.split(delimiter, 1)
                keyword = keyword.strip()
                explanation = explanation.strip()
                keyword_explanation[keyword] = explanation
    return keyword_explanation

@app.route("/keyword")
def keyword():

    key_file = os.path.join(temp_dir,'change_keywords.txt')
    keywords_explanations = read_keyword(key_file)

    return render_template('Outline.html', keyword_explanation=keywords_explanations)

threading.Thread(target=app.run,kwargs={"use_reloader":False,"debug":True}).start()

 * ngrok tunnel "https://edbe-34-16-143-238.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: on
